<a href="https://colab.research.google.com/github/ForestPearson/CS410-510-NLP-project/blob/lstm-replacement/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import StringLookup
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Embedding

BATCH_SIZE = 64
BUFFER_SIZE = 10000
EPOCHS = 30
DIM = 256
RNN = 1024

path = tf.keras.utils.get_file('combined.txt', 'https://raw.githubusercontent.com/ForestPearson/CS410-510-NLP-project/main/data/combined.txt')

389861/389861 [==============================] - 0s 0us/step


In [3]:
text = open(path, 'rb').read().decode(encoding='utf-8')
print("Length:", len(text))
print(text[:500])

vocab = sorted(set(text))

Length: 389861
ACT I

SCENE I. Rousillon. The COUNT's palace.

Enter BERTRAM, the COUNTESS of Rousillon, HELENA, and LAFEU, all in black
COUNTESS
In delivering my son from me, I bury a second husband.
BERTRAM
And I in going, madam, weep o'er my father's death
anew: but I must attend his majesty's command, to
whom I am now in ward, evermore in subjection.
LAFEU
You shall find of the king a husband, madam; you,
sir, a father: he that so generally is at all times
good must of necessity hold his virtue to you; who


In [4]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

ids_from_chars = StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
vocabSize = len(ids_from_chars.get_vocabulary())

ids = ids_from_chars(chars)
chars = chars_from_ids(ids)


In [5]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [6]:
seq_length = 100
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
#Convert to character indices
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
#Form sequences made up of 100 characters
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [7]:
#Training data creation and target creation using sequences
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.lstm = LSTM(rnn_units,return_sequences=True,return_state=True)
    self.dense = Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.lstm.get_initial_state(x)
    # x, states = self.lstm(x, initial_state=states, training=training)
    x, memory_state, carry_state = self.lstm(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      # return x, states
      return x, memory_state, carry_state
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocabSize,
    embedding_dim=DIM,
    rnn_units=RNN)

In [27]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))
# Directory where the checkpoints will be saved
dir = './data/epochs'
#File names
fileName = os.path.join(dir, "ckpt_{epoch}")
results = tf.keras.callbacks.ModelCheckpoint(filepath=fileName,save_weights_only=True)

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[results])

Epoch 1/30
60/60 [==============================] - 7s 65ms/step - loss: 3.3690
Epoch 2/30
60/60 [==============================] - 5s 69ms/step - loss: 2.6806
Epoch 3/30
60/60 [==============================] - 5s 68ms/step - loss: 2.3456
Epoch 4/30
60/60 [==============================] - 5s 69ms/step - loss: 2.1914
Epoch 5/30
60/60 [==============================] - 5s 67ms/step - loss: 2.0527
Epoch 6/30
60/60 [==============================] - 4s 67ms/step - loss: 1.9346
Epoch 7/30
60/60 [==============================] - 4s 68ms/step - loss: 1.8349
Epoch 8/30
60/60 [==============================] - 5s 69ms/step - loss: 1.7532
Epoch 9/30
60/60 [==============================] - 5s 69ms/step - loss: 1.6821
Epoch 10/30
60/60 [==============================] - 5s 68ms/step - loss: 1.6255
Epoch 11/30
60/60 [==============================] - 4s 68ms/step - loss: 1.5740
Epoch 12/30
60/60 [==============================] - 4s 68ms/step - loss: 1.5293
Epoch 13/30
60/60 [==================

In [30]:
class Generate(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  def predict(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # predicted_logits, states = self.model(inputs=input_ids, states=states,return_state=True)
    predicted_logits, memory_state, carry_state = self.model(inputs=input_ids, states=states,return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    # return predicted_chars, states
    return predicted_chars, memory_state, carry_state

In [31]:
Generator = Generate(model, chars_from_ids, ids_from_chars)
states = None
seed = tf.constant(['COUNTESS'])
result = [seed]

for n in range(1000):
  # seed, states = Generator.predict(seed, states=states)
  seed, memory_state, carry_state = Generator.predict(seed, states=states)
  result.append(seed)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

COUNTESS

Figey ayo mat.

PSin if otle hey I then d: n:
PANDZime
S
G-] t LINAn had cos
he ives t bry t
perer-tllllothid bI cus canon iousamo whas.
Hele
G
AMitheverducalonseveath chive
I't w nd d
Win nce u o the wie mixe, chemoungayo'sivecke. I'seate atha yoshts golor:
S
Congala'ttong thyithe icust f tawistoughe myongomaves this!
M[Shainke
PS
ELYCELLI'llear buprms
S
SAThin
LOShafawithe readyok om ar ourdmau cheacares ley f I'ld anoupawertr hat Thine ar h t.
payoranken ie, tathed toow nsse ayonite t bie ayoumoufoun cowx the has s cth yorkyomes edifere ber thengal gr empit,
HI wewin ate
Sh cu d fapre, t d sen fl,
I an omaundgato falll thedsspan, soo ome
Th I cis
O'd tomo
HS
PAUKI'teatond JANUTr, n imeele mange w.
aremprd
HES
I't whostr: oud mard hered 'dyonf.
Whaly owns.
Shid farond oumoruso'LERomawice
Cawit, alaind
DINELours INKn-pearat fazeeayoure stur an test msh he modou I y momy, ssosiowhal ghainfere.
S
IUYUS
EShe sertay me il dechenf bee bLighinge dedemacrco s po'shteead me tos inco